The purposes of this notebook is to use PySpark to extract data from parquet files as external tables, and then convert those external tables into csv for analysis in Pandas.

In [ ]:
#instantiate PySpark in the terminal
! docker-compose exec spark env
! PYSPARK_DRIVER_PYTHON=jupyter PYSPARK_DRIVER_PYTHON_OPTS='notebook --no-browser --port 5000 --ip 0.0.0.0 --allow-root' pyspark

The above configuration will look different. We had to reconfigure our GCP to allow external traffic through port 5000. This was the only way we could connect with a PySpark notebook, hosted on GCP, with our local browsers. 

Now, start creating external tables from the written parquet files, locations include: 

-/tmp/play_hrs
-/tmp/purchase_sword
-/tmp/ref_count
-/tmp/sub_count
-/tmp/money_paid
-/tmp/account_open
-/tmp/join_guild
-/tmp/sw_a_g

In [ ]:
#Query 1, we need hours played and account_open

#create temp tables for both variables
hours = spark.read.parquet('/tmp/play_hrs')
hours.registerTempTable('hours')
acct_open = spark.read.parquet('/tmp/account_open')
acct_open.registerTempTable('open')

#now join tables on Player ID

hrs_open = spark.sql(
    "select * from hrs_open \
    from hours \
    inner join acct_open \
    on hours.host = open.host"
)

#export to df for analysis
df_q1 = hrs_open.toPandas()


In [ ]:
#Query 2, need friend referrals and to join with play time

#create temp table 
ref = spark.read.parquet('/tmp/ref_count')
ref.registerTempTable('ref')

#inner join
hrs_op_ref = spark.sql(
    "select * from hours_open \
    inner join ref \
    on hours_open.host = ref.host"
)

#export to df for analysis
df_q2 = hrs_op_ref.toPandas()


In [ ]:
#Query 3, need money and subscriptions, will join to hrs_op_ref in case

#create temp tables
sub = spark.read.parquet('/tmp/sub_count')
sub.registerTempTable('sub')
money = spark.read.parquet('/tmp/money_paid')
money.registerTempTable('money')

#join all 3 tables
hrs_op_ref_mon_sub = spark.sql(
    "select * \
    from ((hrs_op_ref \
    inner join sub on hrs_op_ref.host = sub.host) \
    inner join money on hrs_op_ref.host = money.host)"
)

#export to df for analysis
df_q3 = hrs_op_ref_mon_sub.toPandas()